Load Vol and Price Data

In [7]:
# Define ticker mapping
import yfinance as yf
import os
import pandas as pd
import numpy as np

tickers = {
    'ES': 'ES=F',
    'CL': 'CL=F',
    'ZN': 'ZN=F',
    '6E': '6E=F'
}

# Download each instrument's OHLCV into its own DataFrame
price_data = {}
for name, symbol in tickers.items():
    try:
        df = yf.download(symbol, start="1980-01-01", auto_adjust=True)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(1)
        df.columns.name = None  # Remove 'Ticker' column level
        df.index.name = None  # Remove index name for cleaner display
        if not df.empty:
            globals()[name.replace(' ', '_').replace('/', '_')] = df
            price_data[name] = df
    except Exception as e:
        print(f"Failed to download {name}: {e}")

# Calculate and store Realized Volatility (20-day rolling std of log returns)
vol_data = {}

for name, symbol in tickers.items():
    try:
        df = yf.download(symbol, start="1980-01-01", auto_adjust=True)
        if df.empty:
            print(f"⚠️ No data for {name}, skipping.")
            continue
        df = df[['Close']].dropna()
        df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
        df['Close_Vol'] = df['Log_Return'].rolling(window=20).std()
        result = df[['Close_Vol']].dropna()
        vol_data[name.replace('/', '_')] = result
    except Exception as e:
        print(f"❌ Failed to calculate vol for {name}: {e}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
c:\Users\owen.osagiede\AppData\Local\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Apply Feature Engineering

In [8]:
import pywt
import numpy as np
import pandas as pd
import os

# --- List of DataFrames to Process ---
enriched_dataframes = {}
all_data = {**price_data, **vol_data}

for label, df in all_data.items():
    try:
        suffix = "" if not label.endswith("_Realized_Vol") else "_rv"
        series = df['Close'] if 'Close' in df.columns else df['Close_Vol']
        features = pd.DataFrame(index=series.index)

        # Date Features
        features[f'month{suffix}'] = series.index.month
        features[f'day_of_week{suffix}'] = series.index.dayofweek
        features[f'month_sin{suffix}'] = np.sin(2 * np.pi * series.index.month / 12)
        features[f'month_cos{suffix}'] = np.cos(2 * np.pi * series.index.month / 12)
        features[f'dow_sin{suffix}'] = np.sin(2 * np.pi * series.index.dayofweek / 7)
        features[f'dow_cos{suffix}'] = np.cos(2 * np.pi * series.index.dayofweek / 7)

        # Basic Indicators
        features[f'log_return{suffix}'] = np.log(series / series.shift(1))
        features[f'ma_50{suffix}'] = series.rolling(window=50).mean()
        features[f'ma_200{suffix}'] = series.rolling(window=200).mean()
        features[f'ma_trend_signal{suffix}'] = np.sign(features[f'ma_50{suffix}'] - features[f'ma_200{suffix}'])
        features[f'price_diff_50{suffix}'] = series - features[f'ma_50{suffix}']
        features[f'price_diff_200{suffix}'] = series - features[f'ma_200{suffix}']

        # MACD
        ema_12 = series.ewm(span=12, adjust=False).mean()
        ema_26 = series.ewm(span=26, adjust=False).mean()
        features[f'macd{suffix}'] = ema_12 - ema_26
        features[f'macd_signal{suffix}'] = features[f'macd{suffix}'].ewm(span=9, adjust=False).mean()
        features[f'macd_binary{suffix}'] = np.sign(features[f'macd{suffix}'] - features[f'macd_signal{suffix}'])

        # Bollinger Bands
        sma = series.rolling(window=20).mean()
        std = series.rolling(window=20).std()
        upper = sma + 2 * std
        lower = sma - 2 * std
        features[f'bb_upper{suffix}'] = upper
        features[f'bb_lower{suffix}'] = lower
        features[f'bb_width{suffix}'] = upper - lower
        features[f'bb_position{suffix}'] = (series - lower) / (upper - lower)

        # Fourier and Wavelet
        features[f'fourier_power{suffix}'] = series.rolling(64).apply(lambda x: np.abs(np.fft.fft(x))[1] if len(x) >= 2 else np.nan, raw=True)
        features[f'wavelet_detail{suffix}'] = series.rolling(64).apply(lambda x: pywt.dwt(x, 'haar')[1][-1] if len(x) >= 2 else np.nan, raw=True)

        # Hurst Exponent
        def compute_hurst(ts):
            lags = range(2, 20)
            tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
            poly = np.polyfit(np.log(lags), np.log(tau), 1)
            return poly[0] if not np.isnan(poly[0]) else np.nan
        features[f'hurst_exponent{suffix}'] = series.rolling(window=100).apply(compute_hurst, raw=False)

        if 'High' in df.columns and 'Low' in df.columns:
            delta = series.diff()
            gain = delta.where(delta > 0, 0.0)
            loss = -delta.where(delta < 0, 0.0)
            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()
            rs = avg_gain / avg_loss
            features[f'rsi{suffix}'] = 100 - (100 / (1 + rs))
            features[f'rsi_threshold{suffix}'] = features[f'rsi{suffix}'].apply(lambda x: 1 if x > 70 else (-1 if x < 30 else 0))

            tp = (df['High'] + df['Low'] + series) / 3
            sma_tp = tp.rolling(window=20).mean()
            mad = tp.rolling(window=20).apply(lambda x: np.mean(np.abs(x - np.mean(x))))
            features[f'cci{suffix}'] = (tp - sma_tp) / (0.015 * mad)
            features[f'cci_threshold{suffix}'] = features[f'cci{suffix}'].apply(lambda x: 1 if x > 100 else (-1 if x < -100 else 0))

            prev_close = series.shift(1)
            tr = pd.concat([
                df['High'] - df['Low'],
                (df['High'] - prev_close).abs(),
                (df['Low'] - prev_close).abs()
            ], axis=1).max(axis=1)
            features[f'atr{suffix}'] = tr.rolling(window=14).mean()

            plus_dm = df['High'].diff()
            minus_dm = df['Low'].diff().abs()
            plus_dm[plus_dm < minus_dm] = 0
            minus_dm[minus_dm < plus_dm] = 0
            tr1 = df['High'] - df['Low']
            tr2 = (df['High'] - series.shift()).abs()
            tr3 = (df['Low'] - series.shift()).abs()
            tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
            atr_tr = tr.rolling(window=14).mean()
            plus_di = 100 * (plus_dm.rolling(window=14).mean() / atr_tr)
            minus_di = 100 * (minus_dm.rolling(window=14).mean() / atr_tr)
            dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di)
            features[f'adx{suffix}'] = dx.rolling(window=14).mean()

            daily_return = series.pct_change()
            obv = df['Volume'].copy()
            obv[series.diff() < 0] *= -1
            obv[series.diff() == 0] = 0
            features[f'obv{suffix}'] = obv.cumsum()
            features[f'vpt{suffix}'] = (daily_return * df['Volume']).cumsum()

            features[f'parkinson_vol{suffix}'] = (1 / (4 * np.log(2))) * (np.log(df['High'] / df['Low']) ** 2).rolling(window=10).mean().apply(np.sqrt)

        # Higher Moments and Rate of Change applied to ALL features
        kurt_df = features.rolling(window=21).kurt().add_suffix('_kurtosis')
        skew_df = features.rolling(window=21).skew().add_suffix('_skew')
        features = pd.concat([features, kurt_df, skew_df], axis=1)

        roc_1d = features.pct_change(1).add_suffix('_roc_1d')
        roc_5d = features.pct_change(5).add_suffix('_roc_5d')
        roc_30d = features.pct_change(30).add_suffix('_roc_30d')
        features = pd.concat([features, roc_1d, roc_5d, roc_30d], axis=1)

        enriched_dataframes[label] = features.dropna(thresh=int(features.shape[1] * 0.8))

    except Exception as e:
        print(f"Feature computation failed for {label}: {e}")

# Combine Price and Realized Vol FE Time Series
merged_enriched = {}

# Group instruments by base name (e.g., 'Coffee' and 'Coffee_Realized_Vol')
all_labels = list(enriched_dataframes.keys())
base_names = set(label.replace("_Realized_Vol", "") for label in all_labels)

for base in base_names:
    related = [enriched_dataframes[k] for k in all_labels if k == base or k == f"{base}_Realized_Vol"]
    if related:
        try:
            merged_enriched[base] = pd.concat(related, axis=1).dropna(thresh=int(0.90 * sum([df.shape[1] for df in related])))
        except Exception as e:
            print(f"Failed to merge features for {base}: {e}")


# Generate pairwise interaction features with correlation filter < 0.50 (absolute value)
interaction_enriched = {}

for label, df in merged_enriched.items():
    try:
        top_n = 50
        top_features = df.var().sort_values(ascending=False).head(top_n).index

        corr_matrix = df[top_features].corr().abs()
        interaction_features = pd.DataFrame(index=df.index)

        for i in range(len(top_features)):
            for j in range(i + 1, len(top_features)):
                col1 = top_features[i]
                col2 = top_features[j]
                if corr_matrix.loc[col1, col2] < 0.5:
                    name = f'{col1}_x_{col2}'
                    interaction_features[name] = df[col1] * df[col2]

        combined = pd.concat([df, interaction_features], axis=1)
        # Only drop rows where more than 10% of columns are missing
        interaction_enriched[label] = combined.dropna(thresh=int(0.9 * combined.shape[1]))

    except Exception as e:
        print(f"Failed to compute interaction features for {label}: {e}")


# Apply EWMA standardization to all interaction-enriched dataframes
normalized_enriched = {}

for label, df in interaction_enriched.items():
    try:
        mean_ewma = df.ewm(span=882, adjust=False).mean()
        std_ewma = df.ewm(span=882, adjust=False).std()
        normalized_enriched[label] = (df - mean_ewma) / std_ewma
    except Exception as e:
        print(f"Failed to apply EWMA normalization for {label}: {e}")
    except Exception as e:
        print(f"Failed to compute interaction features for {label}: {e}")

c:\Users\owen.osagiede\AppData\Local\anaconda3\Lib\site-packages\pandas\core\window\rolling.py:611: RuntimeWarning: All-NaN slice encountered
  return func(x, start, end, min_periods, *numba_args)
C:\Users\owen.osagiede\AppData\Local\Temp\ipykernel_35712\2831393607.py:113: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  roc_1d = features.pct_change(1).add_suffix('_roc_1d')
C:\Users\owen.osagiede\AppData\Local\Temp\ipykernel_35712\2831393607.py:114: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  roc_5d = features.pct_change(5).add_suffix('_roc_5d')
C:\Users\owen.osagiede\AppData\Local\Temp\ipy

Feature Selection

In [14]:
final_features_dict = {
    "ES": [
        "bb_upper",
        "price_diff_50",
        "bb_lower",
        "macd_signal",
        "month_cos_roc_30d_x_month_sin_roc_30d",
        "fourier_power_kurtosis_roc_30d_x_ma_200_skew_roc_30d"
    ],
    "CL": [
        "ma_50",
        "macd",
        "month_cos_roc_5d_x_macd_kurtosis_roc_30d",
        "bb_lower",
        "day_of_week_kurtosis"
    ],
    "ZN": [
        "month_cos_roc_30d_x_bb_upper_kurtosis_roc_5d"
    ],
    "6E": [
        "ma_trend_signal",
        "price_diff_200",
        "price_diff_50_skew_roc_30d_x_bb_position_skew_roc_5d",
        "macd_signal_skew_roc_5d_x_bb_lower_skew_roc_1d",
        "dow_sin_kurtosis",
        "ma_200_skew_roc_30d_x_macd_signal_skew_roc_30d",
        "dow_cos_roc_30d",
        "bb_lower",
        "ma_trend_signal_roc_5d"
    ]
}

# Filter features by label
filtered_features = {}
for label, df in normalized_enriched.items():
    if label in final_features_dict:
        selected_cols = final_features_dict[label]
        filtered_features[label] = df[selected_cols].copy()
    else:
        print(f"\u26a0\ufe0f No selected features found for label: {label}")

Lag Features

In [15]:
lags = [1, 5, 21]
final_features = {}

for label, base_df in filtered_features.items():
    lagged_frames = [base_df]
    for lag in lags:
        lagged = base_df.shift(lag).add_suffix(f"_lag{lag}")
        lagged_frames.append(lagged)

    final_df = pd.concat(lagged_frames, axis=1).dropna()
    final_features[label] = final_df

    print(f"{label}: {final_df.shape[1]} features after lagging")

ZN: 4 features after lagging
6E: 36 features after lagging
ES: 24 features after lagging
CL: 20 features after lagging


HMM + LSTM

In [16]:
import os
import numpy as np
import pandas as pd
import joblib
from keras.models import load_model

lookback = 30
output_dir = "Predicted Regimes"
os.makedirs(output_dir, exist_ok=True)

for label, df in final_features.items():
    print(f"🔄 Processing {label}")

    # Ensure datetime index
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # --- Load HMM Model ---
    hmm_path = f"Models/HMM_{label}.pkl"
    if not os.path.exists(hmm_path):
        print(f"⚠️ HMM model missing for {label}, skipping.")
        continue

    try:
        hmm_model = joblib.load(hmm_path)
        df["HMM_State"] = hmm_model.predict(df)
    except Exception as e:
        print(f"❌ HMM prediction failed for {label}: {e}")
        continue

    # --- Load LSTM Model ---
    lstm_path = f"Models/LSTM_{label}.h5"
    if not os.path.exists(lstm_path):
        print(f"⚠️ LSTM model missing for {label}, skipping.")
        continue

    lstm_model = load_model(lstm_path)

    # --- Build rolling sequences for LSTM ---
    sequences = []
    timestamps = []
    for i in range(lookback, len(df)):
        window = df.iloc[i - lookback:i].values
        sequences.append(window)
        timestamps.append(df.index[i])

    if not sequences:
        print(f"⚠️ Not enough data for LSTM lookback in {label}, skipping.")
        continue

    X_lstm = np.array(sequences)

    try:
        lstm_probs = lstm_model.predict(X_lstm, verbose=0)
        lstm_preds = np.argmax(lstm_probs, axis=1)
    except Exception as e:
        print(f"❌ LSTM prediction failed for {label}: {e}")
        continue

    # --- Create final export ---
    pred_df = pd.DataFrame({
        "Date": timestamps,
        "Vol Regime": lstm_preds
    })

    pred_df.to_csv(f"{output_dir}/PredictedVol_{label}.csv", index=False)
    print(f"✅ Saved LSTM Vol Regime → PredictedVol_{label}.csv")


🔄 Processing ZN


✅ Saved LSTM Vol Regime → PredictedVol_ZN.csv
🔄 Processing 6E


✅ Saved LSTM Vol Regime → PredictedVol_6E.csv
🔄 Processing ES


✅ Saved LSTM Vol Regime → PredictedVol_ES.csv
🔄 Processing CL
✅ Saved LSTM Vol Regime → PredictedVol_CL.csv
